# Setting up data and env

## Upload Your Project

In [ ]:
from google.colab import files
print("Upload your project645.zip file...")
uploaded = files.upload()

# Verify upload
for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')

Upload your project645.zip file...


Saving project645.zip to project645.zip
User uploaded file "project645.zip" with length 24142187 bytes


## Unzip the project

In [ ]:
!unzip -q project645.zip -d /content/
# Verify unzip by listing contents
!ls /content/project645/
!ls /content/project645/code/

code  LICENSE  mai645.yml  README.md  train_data_bvh
analysis.py
fix_feet.py
generate_training_euler_data.py
generate_training_pos_data.py
generate_training_quad_data.py
plot_euler_loss.py
plot_loss.py
__pycache__
pytorch_train_euler_aclstm_for_calculating_loss_to_device.py
pytorch_train_euler_aclstm.py
pytorch_train_pos_aclstm_for_calculating_loss_to_device.py
pytorch_train_pos_aclstm.py
pytorch_train_quad_aclstm.py
read_bvh_hierarchy.py
read_bvh_hierarchy.pyc
read_bvh.py
read_bvh.pyc
rotation2xyz.py
rotation2xyz.pyc
rotation_conversions.py
synthesize_euler_motion.py
synthesize_pos_motion_original_colab.py
synthesize_pos_motion.py
synthesize_quad_motion_debug.py
synthesize_quad_motion.py
test_encodings.py


## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Create Output Folder

*   **Create an Output Folder in Google Drive:** It's highly recommended to create a dedicated folder in your Google Drive to store the outputs (trained models, generated data/BVH files). For example, you could create a folder named `mai645_project_outputs` directly in your "My Drive".
*   Define a variable in Colab pointing to this Drive folder. **Make sure the path matches the folder you created.**

In [ ]:
# IMPORTANT: Adjust this path if you named your Drive folder differently!
GDRIVE_OUTPUT_DIR = "/content/drive/MyDrive/mai645_project_outputs"
# Create the directory in Drive if it doesn't exist (optional, Colab can create it)
import os
os.makedirs(GDRIVE_OUTPUT_DIR, exist_ok=True)
print(f"Outputs will be saved to: {GDRIVE_OUTPUT_DIR}")

Outputs will be saved to: /content/drive/MyDrive/mai645_project_outputs


## Install Dependencies

In [ ]:
# Install the latest compatible PyTorch, torchvision, torchaudio for this Colab environment
!pip install -q torch torchvision torchaudio

# Install other libraries, specifically targeting numpy in the 2.0.x range.
# This aims to satisfy thinc (>=2.0.0) AND tensorflow/numba (<2.1.0).
!pip install -q contourpy==1.3.1 cycler==0.12.1 fonttools==4.56.0 kiwisolver==1.4.8 matplotlib==3.10.1 "numpy>=2.0.0,<2.1.0" opencv-python==4.11.0.86 packaging==24.2 pyparsing==3.2.3 python-dateutil==2.9.0.post0 six==1.17.0 transforms3d==0.4.2

# Verify installation
!pip show torch torchvision torchaudio transforms3d numpy opencv-python tensorflow numba

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.2/326.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Change the current working directory to where your Python scripts are located

In [ ]:
cd /content/project645/code/

/content/project645/code


# Positional Preprocessing

## Upload preprocessed positional data

In [ ]:
import os
# Ensure GDRIVE_OUTPUT_DIR is defined (should match your Drive output folder)
# GDRIVE_OUTPUT_DIR = "/content/drive/MyDrive/mai645_project_outputs" # Uncomment and adjust if not run in a previous cell

# Define source directories in Google Drive for positional data
GDRIVE_POS_TRAIN_DATA_SOURCE_DIR = os.path.join(GDRIVE_OUTPUT_DIR, "train_data_pos", "salsa")
GDRIVE_POS_RECON_BVH_SOURCE_DIR = os.path.join(GDRIVE_OUTPUT_DIR, "reconstructed_bvh_data_pos", "salsa")

# Define target Colab directories
COLAB_POS_TRAIN_DATA_DIR = "/content/project645/train_data_pos/salsa"
COLAB_POS_RECON_BVH_DIR = "/content/project645/reconstructed_bvh_data_pos/salsa"

# Create target directories in Colab if they don't exist
os.makedirs(COLAB_POS_TRAIN_DATA_DIR, exist_ok=True)
os.makedirs(COLAB_POS_RECON_BVH_DIR, exist_ok=True)

print(f"Attempting to copy from {GDRIVE_POS_TRAIN_DATA_SOURCE_DIR} to {COLAB_POS_TRAIN_DATA_DIR}...")
if os.path.exists(GDRIVE_POS_TRAIN_DATA_SOURCE_DIR) and os.listdir(GDRIVE_POS_TRAIN_DATA_SOURCE_DIR):
  get_ipython().system(f"cp -r {GDRIVE_POS_TRAIN_DATA_SOURCE_DIR}/* {COLAB_POS_TRAIN_DATA_DIR}/")
  print(f"Copying training data from Drive complete. Check: !ls {COLAB_POS_TRAIN_DATA_DIR}")
else:
  print(f"Source directory {GDRIVE_POS_TRAIN_DATA_SOURCE_DIR} is empty or does not exist in Google Drive. Skipping copy.")

print(f"Attempting to copy from {GDRIVE_POS_RECON_BVH_SOURCE_DIR} to {COLAB_POS_RECON_BVH_DIR}...")
if os.path.exists(GDRIVE_POS_RECON_BVH_SOURCE_DIR) and os.listdir(GDRIVE_POS_RECON_BVH_SOURCE_DIR):
  get_ipython().system(f"cp -r {GDRIVE_POS_RECON_BVH_SOURCE_DIR}/* {COLAB_POS_RECON_BVH_DIR}/")
  print(f"Copying reconstructed BVH data from Drive complete. Check: !ls {COLAB_POS_RECON_BVH_DIR}")
else:
  print(f"Source directory {GDRIVE_POS_RECON_BVH_SOURCE_DIR} is empty or does not exist in Google Drive. Skipping copy.")

print("Process to load positional data from Google Drive finished.")
# You can verify by listing contents in Colab:
# !ls /content/project645/train_data_pos/salsa/
# !ls /content/project645/reconstructed_bvh_data_pos/salsa/

Attempting to copy from /content/drive/MyDrive/mai645_project_outputs/train_data_pos/salsa to /content/project645/train_data_pos/salsa...
Copying training data from Drive complete. Check: !ls /content/project645/train_data_pos/salsa
Attempting to copy from /content/drive/MyDrive/mai645_project_outputs/reconstructed_bvh_data_pos/salsa to /content/project645/reconstructed_bvh_data_pos/salsa...
Copying reconstructed BVH data from Drive complete. Check: !ls /content/project645/reconstructed_bvh_data_pos/salsa
Process to load positional data from Google Drive finished.


## (19m) Preprocess Positional data

In [ ]:
# Preprocess Positional data
!python generate_training_pos_data.py
# Check if output folders were created for salsa
!ls /content/project645/train_data_pos/
!ls /content/project645/train_data_pos/salsa/
!ls /content/project645/reconstructed_bvh_data_pos/salsa/

salsa
01.bvh.npy  06.bvh.npy	11.bvh.npy  16.bvh.npy	21.bvh.npy  26.bvh.npy
02.bvh.npy  07.bvh.npy	12.bvh.npy  17.bvh.npy	22.bvh.npy  27.bvh.npy
03.bvh.npy  08.bvh.npy	13.bvh.npy  18.bvh.npy	23.bvh.npy  28.bvh.npy
04.bvh.npy  09.bvh.npy	14.bvh.npy  19.bvh.npy	24.bvh.npy  29.bvh.npy
05.bvh.npy  10.bvh.npy	15.bvh.npy  20.bvh.npy	25.bvh.npy  30.bvh.npy
01.bvh.npy.bvh	07.bvh.npy.bvh	13.bvh.npy.bvh	19.bvh.npy.bvh	25.bvh.npy.bvh
02.bvh.npy.bvh	08.bvh.npy.bvh	14.bvh.npy.bvh	20.bvh.npy.bvh	26.bvh.npy.bvh
03.bvh.npy.bvh	09.bvh.npy.bvh	15.bvh.npy.bvh	21.bvh.npy.bvh	27.bvh.npy.bvh
04.bvh.npy.bvh	10.bvh.npy.bvh	16.bvh.npy.bvh	22.bvh.npy.bvh	28.bvh.npy.bvh
05.bvh.npy.bvh	11.bvh.npy.bvh	17.bvh.npy.bvh	23.bvh.npy.bvh	29.bvh.npy.bvh
06.bvh.npy.bvh	12.bvh.npy.bvh	18.bvh.npy.bvh	24.bvh.npy.bvh	30.bvh.npy.bvh


## Save Positional Data to Drive & Download (Optional)

In [ ]:
# Ensure GDRIVE_OUTPUT_DIR is defined
# GDRIVE_OUTPUT_DIR = "/content/drive/MyDrive/mai645_project_outputs" # Uncomment and adjust if not run in a previous cell

# Define source and target directories for positional data
COLAB_POS_TRAIN_DATA_DIR = "/content/project645/train_data_pos/salsa"
COLAB_POS_RECON_BVH_DIR = "/content/project645/reconstructed_bvh_data_pos/salsa"

GDRIVE_POS_TRAIN_DATA_TARGET_DIR = os.path.join(GDRIVE_OUTPUT_DIR, "train_data_pos", "salsa")
GDRIVE_POS_RECON_BVH_TARGET_DIR = os.path.join(GDRIVE_OUTPUT_DIR, "reconstructed_bvh_data_pos", "salsa")

# Create target directories in Google Drive
os.makedirs(GDRIVE_POS_TRAIN_DATA_TARGET_DIR, exist_ok=True)
os.makedirs(GDRIVE_POS_RECON_BVH_TARGET_DIR, exist_ok=True)

print(f"Copying {COLAB_POS_TRAIN_DATA_DIR} to {GDRIVE_POS_TRAIN_DATA_TARGET_DIR}...")
# Use !cp for shell command execution in Colab
# Make sure the source directory is not empty before copying
# Adding a check here, or ensure your preprocessing script ran successfully
if os.path.exists(COLAB_POS_TRAIN_DATA_DIR) and os.listdir(COLAB_POS_TRAIN_DATA_DIR):
  get_ipython().system(f"cp -r {COLAB_POS_TRAIN_DATA_DIR}/* {GDRIVE_POS_TRAIN_DATA_TARGET_DIR}/")
  print("Copying training data complete.")
else:
  print(f"Source directory {COLAB_POS_TRAIN_DATA_DIR} is empty or does not exist. Skipping copy.")

print(f"Copying {COLAB_POS_RECON_BVH_DIR} to {GDRIVE_POS_RECON_BVH_TARGET_DIR}...")
if os.path.exists(COLAB_POS_RECON_BVH_DIR) and os.listdir(COLAB_POS_RECON_BVH_DIR):
  get_ipython().system(f"cp -r {COLAB_POS_RECON_BVH_DIR}/* {GDRIVE_POS_RECON_BVH_TARGET_DIR}/")
  print("Copying reconstructed BVH data complete.")
else:
  print(f"Source directory {COLAB_POS_RECON_BVH_DIR} is empty or does not exist. Skipping copy.")

# Zip the data for download
ZIP_POS_TRAIN_NAME = "project645_train_data_pos_salsa.zip"
ZIP_POS_RECON_NAME = "project645_reconstructed_bvh_pos_salsa.zip"

print(f"Zipping {COLAB_POS_TRAIN_DATA_DIR} to {ZIP_POS_TRAIN_NAME}...")
if os.path.exists(COLAB_POS_TRAIN_DATA_DIR) and os.listdir(COLAB_POS_TRAIN_DATA_DIR):
    get_ipython().system(f"zip -r -q /content/{ZIP_POS_TRAIN_NAME} {COLAB_POS_TRAIN_DATA_DIR}")
    print(f"Zipping training data complete. Ready for download: {ZIP_POS_TRAIN_NAME}")
    try:
        files.download(f"/content/{ZIP_POS_TRAIN_NAME}")
    except NameError: # files might not be imported if this cell is run standalone
        from google.colab import files
        files.download(f"/content/{ZIP_POS_TRAIN_NAME}")
else:
    print(f"Source directory {COLAB_POS_TRAIN_DATA_DIR} is empty or does not exist. Skipping zip and download.")


print(f"Zipping {COLAB_POS_RECON_BVH_DIR} to {ZIP_POS_RECON_NAME}...")
if os.path.exists(COLAB_POS_RECON_BVH_DIR) and os.listdir(COLAB_POS_RECON_BVH_DIR):
    get_ipython().system(f"zip -r -q /content/{ZIP_POS_RECON_NAME} {COLAB_POS_RECON_BVH_DIR}")
    print(f"Zipping reconstructed BVH data complete. Ready for download: {ZIP_POS_RECON_NAME}")
    try:
        files.download(f"/content/{ZIP_POS_RECON_NAME}")
    except NameError:
        from google.colab import files
        files.download(f"/content/{ZIP_POS_RECON_NAME}")
else:
    print(f"Source directory {COLAB_POS_RECON_BVH_DIR} is empty or does not exist. Skipping zip and download.")

print("Data saving and zipping process finished.")
# You can verify by listing contents in Drive:
# !ls "{GDRIVE_POS_TRAIN_DATA_TARGET_DIR}"
# !ls "{GDRIVE_POS_RECON_BVH_TARGET_DIR}"

Copying /content/project645/train_data_pos/salsa to /content/drive/MyDrive/mai645_project_outputs/train_data_pos/salsa...
Copying training data complete.
Copying /content/project645/reconstructed_bvh_data_pos/salsa to /content/drive/MyDrive/mai645_project_outputs/reconstructed_bvh_data_pos/salsa...
Copying reconstructed BVH data complete.
Zipping /content/project645/train_data_pos/salsa to project645_train_data_pos_salsa.zip...
Zipping training data complete. Ready for download: project645_train_data_pos_salsa.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Zipping /content/project645/reconstructed_bvh_data_pos/salsa to project645_reconstructed_bvh_pos_salsa.zip...
Zipping reconstructed BVH data complete. Ready for download: project645_reconstructed_bvh_pos_salsa.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Data saving and zipping process finished.


# Positional training

In [ ]:

GDRIVE_OUTPUT_DIR = "/content/drive/MyDrive/mai645_project_outputs" # Redefine if needed
POS_DATA_DIR = "/content/project645/train_data_pos/salsa/"
POS_WEIGHTS_DIR = os.path.join(GDRIVE_OUTPUT_DIR, "weights_pos")
POS_TRAIN_BVH_DIR = os.path.join(GDRIVE_OUTPUT_DIR, "train_output_bvh_pos")
os.makedirs(POS_WEIGHTS_DIR, exist_ok=True)
os.makedirs(POS_TRAIN_BVH_DIR, exist_ok=True)

!python pytorch_train_pos_aclstm.py \
    --dances_folder "{POS_DATA_DIR}" \
    --write_weight_folder "{POS_WEIGHTS_DIR}/" \
    --write_bvh_motion_folder "{POS_TRAIN_BVH_DIR}/" \
    --dance_frame_rate 60 \
    --batch_size 32 \
    --in_frame 171 \
    --out_frame 171 \
    --seq_len 100 \
    --total_iterations 100000 \
    --read_weight_path "{POS_WEIGHTS_DIR}/0029000.weight" # Optional: to resume

Resuming training from iteration 29001 (loaded checkpoint: 0029000.weight)
Loading motion files...
30  Motion files loaded
###########iter 0029020######################
loss: 0.0009774743
###########iter 0029040######################
loss: 0.00080658536
###########iter 0029060######################
loss: 0.0008417256
###########iter 0029080######################
loss: 0.0009301309
###########iter 0029100######################
loss: 0.0009427726
###########iter 0029120######################
loss: 0.0008720554
###########iter 0029140######################
loss: 0.00082707556
###########iter 0029160######################
loss: 0.00096153526
###########iter 0029180######################
loss: 0.00079231977
###########iter 0029200######################
loss: 0.0008679269
###########iter 0029220######################
loss: 0.0008385109
###########iter 0029240######################
loss: 0.0007683119
###########iter 0029260######################
loss: 0.0007588729
###########iter 0029280#####

# Positional Synthesis

In [ ]:
GDRIVE_OUTPUT_DIR = "/content/drive/MyDrive/mai645_project_outputs" # Redefine if needed
POS_DATA_DIR = "/content/project645/train_data_pos/salsa/"
POS_WEIGHTS_DIR = os.path.join(GDRIVE_OUTPUT_DIR, "weights_pos")
POS_TRAIN_BVH_DIR = os.path.join(GDRIVE_OUTPUT_DIR, "train_output_bvh_pos")
os.makedirs(POS_WEIGHTS_DIR, exist_ok=True)
os.makedirs(POS_TRAIN_BVH_DIR, exist_ok=True)

POS_SYNTHESIS_BVH_DIR = os.path.join(GDRIVE_OUTPUT_DIR, "synthesis_output_bvh_pos")
os.makedirs(POS_SYNTHESIS_BVH_DIR, exist_ok=True)

# IMPORTANT: Replace 'YOUR_ACTUAL_WEIGHT_FILE.weight' with the actual name of your weight file.
ACTUAL_WEIGHT_FILE_NAME = "0049000.weight"
PATH_TO_WEIGHTS = os.path.join(POS_WEIGHTS_DIR, ACTUAL_WEIGHT_FILE_NAME)


print(f"Ensure your weight file is at: {PATH_TO_WEIGHTS}")
print(f"If the weight file above is incorrect, please edit ACTUAL_WEIGHT_FILE_NAME in this cell and re-run.")
print(f"Synthesized BVH files (including ground truth for comparison) will be saved to: {POS_SYNTHESIS_BVH_DIR}")
print(f"Quantitative evaluation (MSE for first N generated frames) will be printed to output.")

# Example command to run synthesis:
# Make sure POS_DATA_DIR, POS_SYNTHESIS_BVH_DIR, and PATH_TO_WEIGHTS are correctly defined from previous cells.
# The script will use default values for model architecture (in_frame_size, hidden_size, etc.)
# unless overridden here.
# batch_size is set to 1 for synthesis to typically focus on one detailed output,
# but can be increased if multiple variations from different seeds are desired.
# quantitative_comparison_len determines how many frames are compared for MSE.

# Before running, ensure ACTUAL_WEIGHT_FILE_NAME is correctly set above!
# If PATH_TO_WEIGHTS does not exist, this command will fail.
if os.path.exists(PATH_TO_WEIGHTS):
    print(f"Attempting to run synthesis with weight file: {PATH_TO_WEIGHTS}")
    get_ipython().system(f'''python synthesize_pos_motion.py \\
        --read_weight_path "{PATH_TO_WEIGHTS}" \\
        --dances_folder "{POS_DATA_DIR}" \\
        --write_bvh_motion_folder "{POS_SYNTHESIS_BVH_DIR}/" \\
        --dance_frame_rate 60 \\
        --batch_size 10 \\
        --initial_seq_len 100 \\
        --generate_frames_number 400 \\
        --quantitative_comparison_len 20''') # This argument is now used by the script
else:
    print(f"ERROR: Weight file not found at {PATH_TO_WEIGHTS}. Please set ACTUAL_WEIGHT_FILE_NAME and ensure the file exists.")

# After running, check the POS_SYNTHESIS_BVH_DIR in your Google Drive for output BVH files (e.g., out00.bvh, gt00.bvh)
# Also check the Colab output above this cell for the printed MSE value.

Ensure your weight file is at: /content/drive/MyDrive/mai645_project_outputs/weights_pos/0049000.weight
If the weight file above is incorrect, please edit ACTUAL_WEIGHT_FILE_NAME in this cell and re-run.
Synthesized BVH files (including ground truth for comparison) will be saved to: /content/drive/MyDrive/mai645_project_outputs/synthesis_output_bvh_pos
Quantitative evaluation (MSE for first N generated frames) will be printed to output.
Attempting to run synthesis with weight file: /content/drive/MyDrive/mai645_project_outputs/weights_pos/0049000.weight
load 01.bvh.npy
frame number: 2243
load 29.bvh.npy
frame number: 1950
load 18.bvh.npy
frame number: 1831
load 21.bvh.npy
frame number: 1773
load 12.bvh.npy
frame number: 1691
load 24.bvh.npy
frame number: 2576
load 27.bvh.npy
frame number: 1691
load 17.bvh.npy
frame number: 2102
load 11.bvh.npy
frame number: 2109
load 16.bvh.npy
frame number: 2243
load 05.bvh.npy
frame number: 1679
load 06.bvh.npy
frame number: 1773
load 02.bvh.npy
fram

# Positional Synthesis Original

In [ ]:
# This cell runs the synthesis process using the `synthesize_pos_motion_original.py` script.
# It requires the same `ACTUAL_WEIGHT_FILE_NAME` to be set as in the cell above for your modified synthesis.
# Outputs from this original script will be saved in a separate directory.
# """

GDRIVE_OUTPUT_DIR = "/content/drive/MyDrive/mai645_project_outputs" # Redefine if needed
POS_DATA_DIR = "/content/project645/train_data_pos/salsa/"
POS_WEIGHTS_DIR = os.path.join(GDRIVE_OUTPUT_DIR, "weights_pos")
POS_TRAIN_BVH_DIR = os.path.join(GDRIVE_OUTPUT_DIR, "train_output_bvh_pos")
os.makedirs(POS_WEIGHTS_DIR, exist_ok=True)
os.makedirs(POS_TRAIN_BVH_DIR, exist_ok=True)
# Define a new output directory for the original script's synthesis results
# Ensure GDRIVE_OUTPUT_DIR is defined from previous cells
POS_SYNTHESIS_ORIGINAL_BVH_DIR = os.path.join(GDRIVE_OUTPUT_DIR, "synthesis_output_bvh_pos_original")
os.makedirs(POS_SYNTHESIS_ORIGINAL_BVH_DIR, exist_ok=True)

# IMPORTANT: Replace 'YOUR_ACTUAL_WEIGHT_FILE.weight' with the actual name of your weight file.
ACTUAL_WEIGHT_FILE_NAME = "0049000.weight"
PATH_TO_WEIGHTS = os.path.join(POS_WEIGHTS_DIR, ACTUAL_WEIGHT_FILE_NAME)
# PATH_TO_WEIGHTS and POS_DATA_DIR should be defined from previous cells.
# ACTUAL_WEIGHT_FILE_NAME also needs to be set correctly in the cell for "Positional Synthesis"
# (running your synthesize_pos_motion.py) or defined/checked here.

# Verify that essential variables are available
if 'PATH_TO_WEIGHTS' not in locals() or 'POS_DATA_DIR' not in locals() or 'GDRIVE_OUTPUT_DIR' not in locals():
    print("ERROR: PATH_TO_WEIGHTS, POS_DATA_DIR, or GDRIVE_OUTPUT_DIR is not defined. "
          "Please ensure the prerequisite cells (like 'Mount Google Drive', 'Create Output Folder', and 'Positional Synthesis' for your modified script) "
          "have been run or these variables are set.")
else:
    print(f"Using GDRIVE_OUTPUT_DIR: {GDRIVE_OUTPUT_DIR}")
    print(f"Using POS_DATA_DIR: {POS_DATA_DIR}")
    print(f"Ensure your weight file is at: {PATH_TO_WEIGHTS}")
    print(f"If the weight file path above is incorrect, please ensure ACTUAL_WEIGHT_FILE_NAME is correctly set in the 'Positional Synthesis' cell.")
    print(f"Synthesized BVH files from the original script will be saved to: {POS_SYNTHESIS_ORIGINAL_BVH_DIR}")

    if os.path.exists(PATH_TO_WEIGHTS):
        print(f"Attempting to run synthesis with ORIGINAL script (synthesize_pos_motion_original.py) using weight file: {PATH_TO_WEIGHTS}")

        get_ipython().system(f'''python synthesize_pos_motion_original_colab.py \\
            --read_weight_path "{PATH_TO_WEIGHTS}" \\
            --dances_folder "{POS_DATA_DIR}" \\
            --write_bvh_motion_folder "{POS_SYNTHESIS_ORIGINAL_BVH_DIR}/" \\
            --dance_frame_rate 60 \\
            --batch_size 5 \\
            --initial_seq_len 100 \\
            --generate_frames_number 400''') # Model parameters (in_frame_size, hidden_size, etc.) will use script defaults
    else:
        print(f"ERROR: Weight file not found at {PATH_TO_WEIGHTS}. "
              f"Please set ACTUAL_WEIGHT_FILE_NAME in the cell for 'Positional Synthesis' "
              f"and ensure the file exists before running this cell.")

# After running, check the POS_SYNTHESIS_ORIGINAL_BVH_DIR in your Google Drive.
# Example: !ls "{POS_SYNTHESIS_ORIGINAL_BVH_DIR}"

Using GDRIVE_OUTPUT_DIR: /content/drive/MyDrive/mai645_project_outputs
Using POS_DATA_DIR: /content/project645/train_data_pos/salsa/
Ensure your weight file is at: /content/drive/MyDrive/mai645_project_outputs/weights_pos/0049000.weight
If the weight file path above is incorrect, please ensure ACTUAL_WEIGHT_FILE_NAME is correctly set in the 'Positional Synthesis' cell.
Synthesized BVH files from the original script will be saved to: /content/drive/MyDrive/mai645_project_outputs/synthesis_output_bvh_pos_original
Attempting to run synthesis with ORIGINAL script (synthesize_pos_motion_original.py) using weight file: /content/drive/MyDrive/mai645_project_outputs/weights_pos/0049000.weight
load 01.bvh.npy
frame number: 2243
load 29.bvh.npy
frame number: 1950
load 18.bvh.npy
frame number: 1831
load 21.bvh.npy
frame number: 1773
load 12.bvh.npy
frame number: 1691
load 24.bvh.npy
frame number: 2576
load 27.bvh.npy
frame number: 1691
load 17.bvh.npy
frame number: 2102
load 11.bvh.npy
frame numb